# Segmenting and Clustering Neighborhoods in Toronto 

### Coursera Capstone Project 2 Part 3

## 4. Exploring and Clustering Neighborhoods in Toronto

In [1]:
#installing necessary libraries
import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', None)

In [2]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium==0.5.0 --yes
from geopy.geocoders import Nominatim
import folium

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [3]:
CLIENT_ID = 'JPOLP02DOHBVSXKIQMJIP2KYLKDWCTP1BH2R1GBEYDOZ0KNP'
CLIENT_SECRET = 'F2B03MEDWRJZFQCK3XPEMLORUA5ARTT5EGNLGGMCR3M3D3F0'
VERSION = '20190606'
LIMIT = 30

In [4]:
df = pd.read_csv('Toronto Geospatial Data.csv')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [5]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Creating a map of Toronto with the neighborhoods superimposed on it:

In [6]:
tmap = folium.Map(location=[latitude, longitude], zoom_start=10)

for borough, neighborhood, lat, long in zip(df['Borough'], df['Neighborhood'], df['Latitude'], df['Longitude']):
    label = '{},{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       color='blue',
                       popup=label,
                       parse_html=False).add_to(tmap)

tmap

I've chosen to explore Scarborough first, so let's cluster only the neighborhoods in Scarborough and store it in a dataframe called scarB:

In [7]:
scarB = df[df['Borough']=='Scarborough'].reset_index(drop=True)
scarB.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
# getting the geographical coordinates of Scarborough:
address = 'Scarborough, Toronto'
geolocater = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Creating a map that shows only the neighborhoods in Scarborough:

In [9]:
scarBmap = folium.Map(location=[latitude, longitude], zoom_start=12)

for borough, neighborhood, lat, long in zip(scarB['Borough'], scarB['Neighborhood'], scarB['Latitude'], scarB['Longitude']):
    label = '{},{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       color='blue',
                       popup=label,
                       parse_html=False).add_to(scarBmap)

scarBmap

Let's explore Cedarbrae neighborhood in Scarborough:

In [10]:
# getting the name and coordinates from the dataframe scarB
c_lat = scarB.loc[4,'Latitude']
c_long = scarB.loc[4, 'Longitude']
c_name = scarB.loc[4, 'Neighborhood']

Getting the top 100 venues in Cedarbrae within the radius of 1000 m:

In [11]:
limit = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, c_lat, c_long, radius, limit)

# sending GET requests
results = requests.get(url).json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning the json and structuring it into a dataframe:

In [13]:
# getting the relevant parts of the json file and transforming it into a pandas dataframe:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# filtering the columns
filtered_cols = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_cols]

# filtering the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#cleaning the columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,CANBE Foods Inc,Indian Restaurant,43.773546,-79.246082
2,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
3,The Real McCoy Burgers And Pizza,Burger Joint,43.774081,-79.230496
4,Tim Hortons,Coffee Shop,43.775992,-79.232135


In [14]:
print("There are {} venues in Cedarbrae.".format(nearby_venues.shape[0]))

There are 30 venues in Cedarbrae.


Repeating the whole process for all neighborhoods in Scarborough by creating a function that does the job:

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

In [16]:
#applying the function to each neighborhood and storing the result in a new dataframe called scarb_venues:
scarb_venues = getNearbyVenues(names=scarB['Neighborhood'],
                              latitudes=scarB['Latitude'],
                              longitudes=scarB['Longitude'])
scarb_venues.shape

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


(89, 7)

In [17]:
# Taking a look at the first 5 venues 
scarb_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


Let's analyze each neighborhood in Scarborough. We use one hot encoding to converting all the categorical variables in a form that better fits the algorithm:

In [18]:
scarb_onehot = pd.get_dummies(scarb_venues[['Venue Category']], prefix="", prefix_sep="")

# adding the neighborhood column to scar_onehot
scarb_onehot['Neighborhood'] = scarb_venues['Neighborhood']

fixed_cols = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_cols]

#grouping rows by neighborhood and taking the mean of the frequency of occurence of each category
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()

Let's get the top 10 venues for each neighborhood:

In [19]:
n = 5 # number of top venues

for neigh in scarb_grouped['Neighborhood']:
    print("----"+ neigh +"----")
    temp = scarb_grouped[scarb_grouped['Neighborhood']==neigh].T.reset_index()
    temp.columns = ['venues', 'frequency']
    temp = temp.iloc[1:]
    temp['frequency'] = temp['frequency'].astype(float)
    temp = temp.round({'frequency':2})
    print(temp.sort_values('frequency', ascending=False).reset_index(drop=True).head(n))
    print('\n')  

----Agincourt----
                venues  frequency
0               Lounge       0.25
1       Breakfast Spot       0.25
2       Sandwich Place       0.25
3   Chinese Restaurant       0.25
4  American Restaurant       0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
              venues  frequency
0   Asian Restaurant       0.33
1               Park       0.33
2         Playground       0.33
3           Pharmacy       0.00
4  Korean Restaurant       0.00


----Birch Cliff, Cliffside West----
                  venues  frequency
0        College Stadium       0.25
1  General Entertainment       0.25
2           Skating Rink       0.25
3                   Café       0.25
4            Pizza Place       0.00


----Cedarbrae----
                 venues  frequency
0  Caribbean Restaurant       0.12
1    Athletics & Sports       0.12
2                Bakery       0.12
3                  Bank       0.12
4       Thai Restaurant       0.12


----Clairlea, Golden Mile, Oakrid

Converting the top 10 venues for each neighborhood into a dataframe:

In [20]:
# first writing a function to sort the venues into descending order
def return_most_common_venues(row, n):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:n]

# creating a new dataframe to display the top 10 venues for each neighborhood:
n = 10
indicators = ['st', 'nd', 'rd']

# creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(n):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighs_venues_sorted = pd.DataFrame(columns=columns)
neighs_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], n)

neighs_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Asian Restaurant,Playground,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment


Now let's cluster the neighborhoods using KMeans algorithm:

In [21]:
k = 5 # number of clusters

scarb_clusters = scarb_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=k, 
                random_state=0).fit(scarb_clusters)

# check cluster labels generated for each row in the dataframe:
kmeans.labels_

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 1, 2, 0, 1], dtype=int32)

In [22]:
# adding cluster labels
neighs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarb_merged = scarB

# merging to add latitude and longitude values for each neighborhood
scarb_merged = scarb_merged.join(neighs_venues_sorted.set_index('Neighborhood'),
                                on='Neighborhood')

scarb_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment


In [23]:
scarb_merged.isnull().sum()

PostalCode                0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            1
1st Most Common Venue     1
2nd Most Common Venue     1
3rd Most Common Venue     1
4th Most Common Venue     1
5th Most Common Venue     1
6th Most Common Venue     1
7th Most Common Venue     1
8th Most Common Venue     1
9th Most Common Venue     1
10th Most Common Venue    1
dtype: int64

Notice that there are some null values in scarb_merged. Let's find out which ones are those:

In [24]:
scarb_merged.loc[scarb_merged.isnull().any(axis=1)] 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So there is no data on venues for Upper Rouge, Scarborough. That is why no cluster is assigned to it. Let's drop this neighborhood since it will result in errors when we try to visualize the clusters:

In [25]:
scarb_merged.dropna(subset=['Cluster Labels'], axis=0, inplace=True)
scarb_merged.isnull().sum()

PostalCode                0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

Now let's take a look at our final dataframe:

In [26]:
scarb_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment


The cluster labels are of datatype float, and we need to convert it into integer.

In [27]:
scarb_merged['Cluster Labels'] = scarb_merged['Cluster Labels'].astype(int)
scarb_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment


Everything looks fine now. Let's visualize the clusters on the map:

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, long], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's examine each cluster:

**Cluster 0**

In [29]:
clus0 = scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[2] + list(range(5, scarb_merged.shape[1]))]]
clus0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
14,"Agincourt North, L'Amoreaux East, Milliken, St...",0,Park,Asian Restaurant,Playground,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


The 1st most common venues in the two neighborhoods in Cluster 0 are playgrounds and parks. So we can name this cluster as 'Parks and Playgrounds'.

**Cluster 1**

In [30]:
clus1 = scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[2] + list(range(5, scarb_merged.shape[1]))]]
clus1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,Woburn,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,Cedarbrae,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
6,"East Birchmount Park, Ionview, Kennedy Park",1,Discount Store,Department Store,Playground,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,"Clairlea, Golden Mile, Oakridge",1,Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment
9,"Birch Cliff, Cliffside West",1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,"Dorset Park, Scarborough Town Centre, Wexford ...",1,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Pet Store,Bakery,Grocery Store,General Entertainment,Fried Chicken Joint,Asian Restaurant
11,"Maryvale, Wexford",1,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint
12,Agincourt,1,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,"Clarks Corners, Sullivan, Tam O'Shanter",1,Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Bank,Chinese Restaurant,Pharmacy,Coffee Shop


It's a little difficult to pin a name for Cluster 1 as there are so many different types of venues that are most common. Let's do a value count to find out which venue appears the most number of times for all neighborhoods in the column '1st Most Common Venue':

In [31]:
clus1['1st Most Common Venue'].value_counts()

Bakery                       1
Discount Store               1
Intersection                 1
Lounge                       1
Pizza Place                  1
Chinese Restaurant           1
Middle Eastern Restaurant    1
Indian Restaurant            1
Coffee Shop                  1
General Entertainment        1
Caribbean Restaurant         1
Name: 1st Most Common Venue, dtype: int64

In this result, each venue category has a value of 1. This means that the most common venue in each of the neighborhoods in Cluster 1 is unique. Therefore, it's even harder to pinpoint the most common venue category for this cluster. However, when we take a closer look at Cluster 1 dataframe, we can see that most of the venues are restaurants, or cater food in one way or the other, as bakeries, coffee shops and lounges. So, it is sensible that we can label this cluster as 'Restaurants and Bakeries'.

**Cluster 2**

In [32]:
clus2 = scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[2] + list(range(5, scarb_merged.shape[1]))]]
clus2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


The most common venue for Cluster 2 are fast food restaurants. But since there is only one neighborhood in this cluster, this can easily mislead us. Let's check the mean of the frequency for each venue category:

In [33]:
# getting the neighborhood's row from scarb_grouped which has the frequency data
scarb_grouped.loc[scarb_grouped['Neighborhood']=='Rouge, Malvern']

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
13,"Rouge, Malvern",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We can see that print shops and fast food restaurants have the same frequency of occurence, which is 0.5. Also, since all the other categories are nearly zero, let's call Cluster 2 as 'Print Shops and Fast Food Restaurants'.

**Cluster 3**

In [34]:
clus3 = scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[2] + list(range(5, scarb_merged.shape[1]))]]
clus3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",3,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [35]:
scarb_grouped.loc[scarb_grouped['Neighborhood']=='Highland Creek, Rouge Hill, Port Union']

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
10,"Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Since bar and construction and landscaping have the same frequency of 0.5, let's rename this cluster 'Bars, Construction and Landscaping'

**Cluster 4**

In [36]:
clus4 = scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[2] + list(range(5, scarb_merged.shape[1]))]]
clus4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffcrest, Cliffside, Scarborough Village West",4,Motel,American Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


In [37]:
scarb_grouped.loc[scarb_grouped['Neighborhood']=='Cliffcrest, Cliffside, Scarborough Village West']

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's name Cluster 4 as 'Motels' since it has a frequency of 0.67.